In [2]:
from pathlib import Path
import requests
import json
from datetime import datetime

In [3]:
# Project root is one level above notebooks
ROOT = Path.cwd().parent

DATA_DIR = ROOT / "data"
CACHE_DIR = DATA_DIR / "cache"

DATA_DIR.mkdir(exist_ok=True)
CACHE_DIR.mkdir(exist_ok=True)

print("ROOT:", ROOT)
print("CACHE DIR:", CACHE_DIR)

ROOT: /Users/nicolas/fantasy assistant
CACHE DIR: /Users/nicolas/fantasy assistant/data/cache


In [4]:
FPL_BOOTSTRAP_URL = "https://fantasy.premierleague.com/api/bootstrap-static/"

def fetch_fpl_data():
    """
    Fetches the main FPL bootstrap data from the public API.
    
    Returns:
        dict: Parsed JSON response containing players, teams, and events.
    """
    response = requests.get(FPL_BOOTSTRAP_URL)
    response.raise_for_status()  # raises error if request failed
    return response.json()

In [5]:
data = fetch_fpl_data()

print("Keys in response:")
print(data.keys())

Keys in response:
dict_keys(['chips', 'events', 'game_settings', 'game_config', 'phases', 'teams', 'total_players', 'element_stats', 'element_types', 'elements'])


In [6]:
# Extract the list of players
players = data["elements"]

print("Number of players:", len(players))
print("\nFirst player example:\n")
print(players[0])

Number of players: 817

First player example:

{'can_transact': True, 'can_select': True, 'chance_of_playing_next_round': None, 'chance_of_playing_this_round': None, 'code': 154561, 'cost_change_event': 0, 'cost_change_event_fall': 0, 'cost_change_start': 5, 'cost_change_start_fall': -5, 'dreamteam_count': 1, 'element_type': 1, 'ep_next': '3.3', 'ep_this': '3.8', 'event_points': 0, 'first_name': 'David', 'form': '2.8', 'id': 1, 'in_dreamteam': True, 'news': '', 'news_added': None, 'now_cost': 60, 'photo': '154561.jpg', 'points_per_game': '4.0', 'removed': False, 'second_name': 'Raya Martín', 'selected_by_percent': '34.8', 'special': False, 'squad_number': None, 'status': 'a', 'team': 1, 'team_code': 3, 'total_points': 109, 'transfers_in': 3926102, 'transfers_in_event': 339, 'transfers_out': 2207905, 'transfers_out_event': 1290, 'value_form': '0.5', 'value_season': '18.2', 'web_name': 'Raya', 'known_name': '', 'region': 200, 'team_join_date': '2024-07-04', 'birth_date': '1995-09-15', 'h

In [7]:
# Here I will build a simplified list of player records (mini dataset)

clean_players = []

for p in players:
    clean_players.append({
        "id": p["id"],
        "name": p["web_name"],
        "first_name": p["first_name"],
        "second_name": p["second_name"],
        "team_id": p["team"],
        "position_id": p["element_type"],
        "price": p["now_cost"] / 10,          # FPL stores price as integer tenths
        "total_points": p["total_points"],
        "form": float(p["form"]) if p["form"] else 0.0,
        "minutes": p["minutes"],
        "points_per_game": float(p["points_per_game"]) if p["points_per_game"] else 0.0,
        "selected_by_percent": float(p["selected_by_percent"]) if p["selected_by_percent"] else 0.0,
        "status": p["status"],
    })

print("Clean players:", len(clean_players))
print("Example clean player:\n", clean_players[0])

Clean players: 817
Example clean player:
 {'id': 1, 'name': 'Raya', 'first_name': 'David', 'second_name': 'Raya Martín', 'team_id': 1, 'position_id': 1, 'price': 6.0, 'total_points': 109, 'form': 2.8, 'minutes': 2430, 'points_per_game': 4.0, 'selected_by_percent': 34.8, 'status': 'a'}


In [8]:
# Right now clean_players is just a list of dictionaries.

# I will now:
# Create a real Player class
#    Add attributes properly
#    Add magic methods (__str__, __repr__, __eq__, __hash__)
#    Make players sortable
#    Make them hashable (important for sets + squad rules later)

In [9]:
class Player:
    """
    Represents a Fantasy Premier League player.
    
    This class wraps cleaned API data into a structured object.
    """

    # Class attribute (shared by all players)
    POSITION_MAP = {
        1: "Goalkeeper",
        2: "Defender",
        3: "Midfielder",
        4: "Forward"
    }

    def __init__(self, id, name, team_id, position_id, price,
                 total_points, form, minutes, points_per_game,
                 selected_by_percent, status):
        
        # Instance attributes
        self.id = id
        self.name = name
        self.team_id = team_id
        self.position_id = position_id
        self.price = price
        self.total_points = total_points
        self.form = form
        self.minutes = minutes
        self.points_per_game = points_per_game
        self.selected_by_percent = selected_by_percent
        self.status = status

    @property
    def position(self):
        """Return human-readable position name."""
        return self.POSITION_MAP.get(self.position_id, "Unknown")

    def __str__(self):
        return f"{self.name} ({self.position}) - £{self.price}m"

    def __repr__(self):
        return f"Player(id={self.id}, name={self.name})"

    def __eq__(self, other):
        if isinstance(other, Player):
            return self.id == other.id
        return False

    def __hash__(self):
        return hash(self.id)

    def __lt__(self, other):
        """Allows sorting players by total points."""
        return self.total_points < other.total_points

In [10]:
# Now to test if it worked:
# Convert cleaned player dictionaries into Player objects
player_objects = []

for p in clean_players:
    player_objects.append(
        Player(
            id=p["id"],
            name=p["name"],
            team_id=p["team_id"],
            position_id=p["position_id"],
            price=p["price"],
            total_points=p["total_points"],
            form=p["form"],
            minutes=p["minutes"],
            points_per_game=p["points_per_game"],
            selected_by_percent=p["selected_by_percent"],
            status=p["status"]
        )
    )

print("Number of Player objects:", len(player_objects))
print("Example Player object:")
print(player_objects[0])

Number of Player objects: 817
Example Player object:
Raya (Goalkeeper) - £6.0m


In [11]:
# Now we demonstrate that your Player objects behave like real well-designed objects:

#   Sorting works because of __lt__

#   Sets work because of __hash__ + __eq__

#   We’ll use collections.Counter to count positions

In [12]:
from collections import Counter

# 1) Sorting: get top 10 players by total points
top10 = sorted(player_objects, reverse=True)[:10]

print("Top 10 players by total points:")
for p in top10:
    print(f"- {p.name:15} | {p.position:11} | Points: {p.total_points} | Price: {p.price}")

# 2) Hashing: make a set (removes duplicates if any)
player_set = set(player_objects)
print("\nUnique players in set:", len(player_set))

# 3) Collections: count players per position
pos_counts = Counter([p.position for p in player_objects])
print("\nPlayers per position:", pos_counts)

Top 10 players by total points:
- Haaland         | Forward     | Points: 187 | Price: 14.8
- Semenyo         | Midfielder  | Points: 153 | Price: 8.0
- Rice            | Midfielder  | Points: 148 | Price: 7.6
- B.Fernandes     | Midfielder  | Points: 143 | Price: 9.8
- Gabriel         | Defender    | Points: 142 | Price: 7.1
- João Pedro      | Forward     | Points: 138 | Price: 7.7
- Thiago          | Forward     | Points: 135 | Price: 7.0
- Bruno G.        | Midfielder  | Points: 134 | Price: 6.9
- Chalobah        | Defender    | Points: 130 | Price: 5.8
- Rogers          | Midfielder  | Points: 127 | Price: 7.7

Unique players in set: 817

Players per position: Counter({'Midfielder': 367, 'Defender': 265, 'Goalkeeper': 94, 'Forward': 91})


In [13]:
# A Fantasy Football Assistant needs a Squad that contains players.

#  This step adds:

#     Composition (Squad has Players)

#     Mutable attributes (the squad’s list of players changes)

#     Iterables (you can loop over a squad)

#     Magic methods (__len__, __iter__, __str__)

#     A foundation for rules (budget, max 3 per team, etc.)

In [14]:
class Squad:
    """
    Represents a fantasy squad made of Player objects.
    Demonstrates composition: a Squad contains Players.
    """

    def __init__(self, budget=100.0):
        self.budget = budget                  # immutable-like float
        self.players = []                     # mutable list

    def add_player(self, player):
        """Add a Player to the squad if budget allows."""
        if self.total_cost() + player.price > self.budget:
            raise ValueError("Budget exceeded!")
        self.players.append(player)

    def remove_player(self, player_id):
        """Remove a player by id."""
        self.players = [p for p in self.players if p.id != player_id]

    def total_cost(self):
        """Return total cost of the squad."""
        return sum(p.price for p in self.players)

    def __len__(self):
        return len(self.players)

    def __iter__(self):
        """Allows: for player in squad"""
        return iter(self.players)

    def __str__(self):
        return f"Squad(players={len(self.players)}, cost={self.total_cost():.1f}/{self.budget})"

In [15]:
# Test

squad = Squad(budget=100.0)

# Add 3 players (just as a test)
squad.add_player(player_objects[0])
squad.add_player(player_objects[1])
squad.add_player(player_objects[2])

print(squad)

print("\nPlayers in squad:")
for p in squad:
    print("-", p)


Squad(players=3, cost=14.1/100.0)

Players in squad:
- Raya (Goalkeeper) - £6.0m
- Arrizabalaga (Goalkeeper) - £4.1m
- Hein (Goalkeeper) - £4.0m


In [16]:
# I will simulate a transfer without modifying the original squad.

#   This step covers:

#     Deep copy vs shallow copy

#     Pass-by-reference behavior

#     Mutable objects

#     Copy module

In [17]:
import copy

# Create a shallow copy
shallow_squad = copy.copy(squad)

print("Original squad size:", len(squad))
print("Shallow squad size:", len(shallow_squad))

# Add a player to shallow copy
shallow_squad.add_player(player_objects[3])

print("\nAfter adding to shallow copy:")
print("Original squad size:", len(squad))
print("Shallow squad size:", len(shallow_squad))

Original squad size: 3
Shallow squad size: 3

After adding to shallow copy:
Original squad size: 4
Shallow squad size: 4


In [18]:
# Create deep copy
deep_squad = copy.deepcopy(squad)

print("Original squad size:", len(squad))
print("Deep squad size:", len(deep_squad))

# Add a player to deep copy
deep_squad.add_player(player_objects[4])

print("\nAfter adding to deep copy:")
print("Original squad size:", len(squad))
print("Deep squad size:", len(deep_squad))

Original squad size: 4
Deep squad size: 4

After adding to deep copy:
Original squad size: 4
Deep squad size: 5


In [19]:
#  Now we create a scoring function so the app can recommend players.

# We’ll start simple and defendable:

#   A player’s score will be based on:

#     total_points (overall performance)

#     form (recent performance)

#     points_per_game (consistency)

#     minutes (reliability)

#     price (value)

In [20]:
def player_score(player, w_points=1.0, w_form=8.0, w_ppg=6.0, w_minutes=0.002, w_value=10.0):
    """
    Compute a recommendation score for a player.
    
    Args:
        player (Player): Player object
        w_points, w_form, w_ppg, w_minutes, w_value: weights (default arguments)
        
    Returns:
        float: score
    """
    value = (player.total_points / player.price) if player.price > 0 else 0.0
    return (
        w_points * player.total_points +
        w_form * player.form +
        w_ppg * player.points_per_game +
        w_minutes * player.minutes +
        w_value * value
    )

In [21]:
# Test:

# Create a list of (player, score) pairs
scored_players = [(p, player_score(p)) for p in player_objects]

# Sort by score descending
scored_players.sort(key=lambda x: x[1], reverse=True)

print("Top 10 recommended players (overall):")
for p, s in scored_players[:10]:
    print(f"- {p.name:15} | {p.position:11} | Score: {s:8.2f} | Price: {p.price} | Points: {p.total_points}")

Top 10 recommended players (overall):
- Semenyo         | Midfielder  | Score:   434.95 | Price: 8.0 | Points: 153
- Guéhi           | Defender    | Score:   428.38 | Price: 5.2 | Points: 124
- João Pedro      | Forward     | Score:   425.35 | Price: 7.7 | Points: 138
- Chalobah        | Defender    | Score:   420.67 | Price: 5.8 | Points: 130
- Mukiele         | Defender    | Score:   420.53 | Price: 4.6 | Points: 114
- Gabriel         | Defender    | Score:   416.01 | Price: 7.1 | Points: 142
- Rice            | Midfielder  | Score:   411.81 | Price: 7.6 | Points: 148
- Lacroix         | Defender    | Score:   402.33 | Price: 5.1 | Points: 116
- Haaland         | Forward     | Score:   396.04 | Price: 14.8 | Points: 187
- Matheus N.      | Defender    | Score:   395.37 | Price: 5.4 | Points: 115


In [22]:
# We’ll now create a recommendation function that can filter players by things like:

#    position (MID, FWD…)

#    max price

#    minimum minutes

#    team id

#    We will implement it using:

#    **kwargs (dynamic filters)

#    a generator (yield) (efficient iteration)

In [23]:
POSITION_NAME_TO_ID = {
    "GK": 1,
    "DEF": 2,
    "MID": 3,
    "FWD": 4
}

In [24]:
def recommend_players(players, limit=10, **filters):
    """
    Yield top recommended players based on filters.
    
    Args:
        players (list[Player]): list of Player objects
        limit (int): number of results to yield
        **filters: supported keys:
            position (str): "GK"/"DEF"/"MID"/"FWD"
            max_price (float)
            min_minutes (int)
            team_id (int)
    
    Yields:
        tuple(Player, float): player and their computed score
    """
    filtered = players

    # Apply filters if provided
    if "position" in filters:
        pos_id = POSITION_NAME_TO_ID[filters["position"]]
        filtered = [p for p in filtered if p.position_id == pos_id]

    if "max_price" in filters:
        filtered = [p for p in filtered if p.price <= filters["max_price"]]

    if "min_minutes" in filters:
        filtered = [p for p in filtered if p.minutes >= filters["min_minutes"]]

    if "team_id" in filters:
        filtered = [p for p in filtered if p.team_id == filters["team_id"]]

    # Score and sort
    scored = [(p, player_score(p)) for p in filtered]
    scored.sort(key=lambda x: x[1], reverse=True)

    # Generator: yield results lazily
    for item in scored[:limit]:
        yield item

In [25]:
# Test
print("Top 10 MID under £8.0 with at least 500 minutes:\n")

for p, s in recommend_players(player_objects, limit=10, position="MID", max_price=8.0, min_minutes=500):
    print(f"- {p.name:15} | Score: {s:8.2f} | Price: {p.price} | Minutes: {p.minutes}")

Top 10 MID under £8.0 with at least 500 minutes:

- Semenyo         | Score:   434.95 | Price: 8.0 | Minutes: 2250
- Rice            | Score:   411.81 | Price: 7.6 | Minutes: 2235
- Stach           | Score:   389.54 | Price: 4.7 | Minutes: 1767
- Bruno G.        | Score:   389.44 | Price: 6.9 | Minutes: 2019
- Anderson        | Score:   386.34 | Price: 5.4 | Minutes: 2340
- Wilson          | Score:   381.64 | Price: 5.9 | Minutes: 1925
- Garner          | Score:   374.13 | Price: 5.2 | Minutes: 2333
- Zubimendi       | Score:   373.42 | Price: 5.3 | Minutes: 2265
- Casemiro        | Score:   369.48 | Price: 5.6 | Minutes: 1724
- Gravenberch     | Score:   368.65 | Price: 5.5 | Minutes: 2118


In [26]:
import sys
from pathlib import Path

# Add project root to Python path
ROOT = Path.cwd().parent
sys.path.append(str(ROOT))

In [27]:
# Testing models.py works in src folder
from src.models import Player, Squad

test_squad = Squad(budget=100.0)
test_squad.add_player(player_objects[0])

print(test_squad)
print(test_squad.players[0])

Squad(players=1, cost=6.0/100.0)
Raya (Goalkeeper) - £6.0m


In [28]:
# Testing advisor.py works in src folder

from src.advisor import recommend_players

print("Top 5 MID under £8.0:\n")

for p, s in recommend_players(player_objects, limit=5, position="MID", max_price=8.0):
    print(f"- {p.name} | Score: {s:.2f}")

Top 5 MID under £8.0:

- Semenyo | Score: 434.95
- Rice | Score: 411.81
- Stach | Score: 389.54
- Bruno G. | Score: 389.44
- Anderson | Score: 386.34


In [29]:
# Test API
from src.data_api import fetch_bootstrap_static

data2 = fetch_bootstrap_static(ttl_seconds=3600)
print(data2.keys())

dict_keys(['chips', 'events', 'game_settings', 'game_config', 'phases', 'teams', 'total_players', 'element_stats', 'element_types', 'elements'])


In [30]:
# Test transform

from src.data_api import fetch_bootstrap_static
from src.transform import build_player_objects

data3 = fetch_bootstrap_static()
players3 = build_player_objects(data3)

print("Players:", len(players3))
print("Example:", players3[0])

Players: 817
Example: Raya (Goalkeeper) - £6.0m


In [31]:
## Clean pipeline using src modules

In [32]:
import sys
from pathlib import Path

ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.data_api import fetch_bootstrap_static
from src.transform import build_player_objects
from src.advisor import recommend_players

# Load data and build players
data = fetch_bootstrap_static(ttl_seconds=3600)
player_objects = build_player_objects(data)

print("Loaded players:", len(player_objects))
print("Example:", player_objects[0])

print("\nTop 5 budget midfielders under £8.0:\n")
for p, s in recommend_players(player_objects, limit=5, position="MID", max_price=8.0, min_minutes=500):
    print(f"- {p.name:15} | Score: {s:8.2f} | Price: {p.price}")

Loaded players: 817
Example: Raya (Goalkeeper) - £6.0m

Top 5 budget midfielders under £8.0:

- Semenyo         | Score:   434.95 | Price: 8.0
- Rice            | Score:   411.81 | Price: 7.6
- Stach           | Score:   389.54 | Price: 4.7
- Bruno G.        | Score:   389.44 | Price: 6.9
- Anderson        | Score:   386.34 | Price: 5.4


In [33]:
# Test validators
from src.validators import validate_formation, validate_search_text

print("Formation tests:")
print("3-5-2:", validate_formation("3-5-2"))
print("433:", validate_formation("433"))
print("4-3-3:", validate_formation("4-3-3"))

print("\nSearch tests:")
print("Salah:", validate_search_text("Salah"))
print("De Bruyne:", validate_search_text("De Bruyne"))
print("Bad input !!!:", validate_search_text("Bad input !!!"))

Formation tests:
3-5-2: True
433: False
4-3-3: True

Search tests:
Salah: True
De Bruyne: True
Bad input !!!: False


In [34]:
# Test rules
from src.rules import BudgetRule, FormationRule, MaxFromTeamRule, RuleViolation
from src.models import Squad

# Build a quick test squad
test_squad = Squad(budget=20.0)  # deliberately small budget to trigger rules
test_squad.add_player(player_objects[0])
test_squad.add_player(player_objects[1])

rules = [
    BudgetRule(),
    MaxFromTeamRule(max_per_team=1),
]

for r in rules:
    try:
        r.validate(test_squad)
        print(f"{r.__class__.__name__}: OK")
    except RuleViolation as e:
        print(f"{r.__class__.__name__}: VIOLATION -> {e}")

BudgetRule: OK
MaxFromTeamRule: VIOLATION -> Too many players from team 1: 2 > 1


In [35]:
# Test advisor_engine:

from src.advisor_engine import Advisor
from src.rules import BudgetRule, MaxFromTeamRule

advisor = Advisor(rules=[BudgetRule(), MaxFromTeamRule(max_per_team=3)])

# Validate your earlier squad from step 9 (named `squad`)
print("Squad validation errors:", advisor.validate_squad(squad))

print("\nTop 5 defenders under £6.0:\n")
recs = advisor.top_recommendations(player_objects, limit=5, position="DEF", max_price=6.0, min_minutes=500)
for p, s in recs:
    print(f"- {p.name:15} | Score: {s:8.2f} | Price: {p.price}")

Squad validation errors: ['Too many players from team 1: 4 > 3']

Top 5 defenders under £6.0:

- Guéhi           | Score:   428.38 | Price: 5.2
- Chalobah        | Score:   420.67 | Price: 5.8
- Mukiele         | Score:   420.53 | Price: 4.6
- Lacroix         | Score:   402.33 | Price: 5.1
- Matheus N.      | Score:   395.37 | Price: 5.4


In [36]:
from collections import Counter

# Count players per team in your current squad
team_counts = Counter([p.team_id for p in squad])

print("Players per team in current squad:")
for team_id, count in team_counts.items():
    print(f"- Team {team_id}: {count} players")

# Show which players are in team 1
team_1_players = [p for p in squad if p.team_id == 1]

print("\nPlayers in team 1:")
for p in team_1_players:
    print(f"- {p.id}: {p.name} ({p.position}) £{p.price}m")

Players per team in current squad:
- Team 1: 4 players

Players in team 1:
- 1: Raya (Goalkeeper) £6.0m
- 2: Arrizabalaga (Goalkeeper) £4.1m
- 3: Hein (Goalkeeper) £4.0m
- 4: Setford (Goalkeeper) £3.9m


In [37]:
# Remove Setford (id=4) to fix max-per-team rule
squad.remove_player(4)

print("Updated squad:", squad)

print("\nUpdated squad players:")
for p in squad:
    print("-", p)

# Re-validate
errors = advisor.validate_squad(squad)
print("\nValidation errors after removal:", errors)

Updated squad: Squad(players=3, cost=14.1/100.0)

Updated squad players:
- Raya (Goalkeeper) - £6.0m
- Arrizabalaga (Goalkeeper) - £4.1m
- Hein (Goalkeeper) - £4.0m

Validation errors after removal: []


In [38]:
from src.rules import FormationRule, RuleViolation
from src.models import Squad

# Create a new squad for a valid starting XI
xi = Squad(budget=100.0)

# Pick a formation (DEF-MID-FWD)
formation = "4-4-2"
rule = FormationRule(formation)

# Helper: get top players by position
def top_by_position(position_id, n):
    candidates = [p for p in player_objects if p.position_id == position_id and p.status == "a"]
    # sort by total_points (uses __lt__ in Player)
    return sorted(candidates, reverse=True)[:n]

# Build XI: 1 GK, 4 DEF, 4 MID, 2 FWD
for p in top_by_position(1, 1):  # GK
    xi.add_player(p)

for p in top_by_position(2, 4):  # DEF
    xi.add_player(p)

for p in top_by_position(3, 4):  # MID
    xi.add_player(p)

for p in top_by_position(4, 2):  # FWD
    xi.add_player(p)

print("XI built:", xi)

# Validate formation
try:
    rule.validate(xi)
    print("Formation valid:", formation)
except RuleViolation as e:
    print("Formation violation:", e)

print("\nStarting XI:")
for p in xi:
    print("-", p, "| Points:", p.total_points)

XI built: Squad(players=11, cost=86.1/100.0)
Formation valid: 4-4-2

Starting XI:
- Raya (Goalkeeper) - £6.0m | Points: 109
- Gabriel (Defender) - £7.1m | Points: 142
- Chalobah (Defender) - £5.8m | Points: 130
- J.Timber (Defender) - £6.4m | Points: 124
- Guéhi (Defender) - £5.2m | Points: 124
- Semenyo (Midfielder) - £8.0m | Points: 153
- Rice (Midfielder) - £7.6m | Points: 148
- B.Fernandes (Midfielder) - £9.8m | Points: 143
- Rogers (Midfielder) - £7.7m | Points: 127
- Haaland (Forward) - £14.8m | Points: 187
- João Pedro (Forward) - £7.7m | Points: 138


In [39]:
from src.models import Squad
from src.rules import FormationRule, RuleViolation

def build_starting_xi(players, advisor, formation="4-4-2", budget=100.0):
    """
    Build a starting XI using advisor scores under a formation constraint.

    Args:
        players: list[Player]
        advisor: Advisor
        formation: 'DEF-MID-FWD' string like '4-4-2'
        budget: float

    Returns:
        Squad: a Squad object representing the starting XI
    """
    rule = FormationRule(formation)
    def_count, mid_count, fwd_count = [int(x) for x in formation.split("-")]

    xi = Squad(budget=budget)

    def top_scored(position_id, n):
        candidates = [p for p in players if p.position_id == position_id and p.status == "a"]
        scored = [(p, advisor.score(p)) for p in candidates]
        scored.sort(key=lambda x: x[1], reverse=True)
        return [p for p, _ in scored[:n]]

    # 1 GK
    for p in top_scored(1, 1):
        xi.add_player(p)

    # DEF/MID/FWD from formation
    for p in top_scored(2, def_count):
        xi.add_player(p)

    for p in top_scored(3, mid_count):
        xi.add_player(p)

    for p in top_scored(4, fwd_count):
        xi.add_player(p)

    # Validate formation
    rule.validate(xi)
    return xi


# Test it
xi2 = build_starting_xi(player_objects, advisor, formation="4-4-2", budget=100.0)

print("XI built with advisor score:", xi2)
print("\nPlayers in XI:")
for p in xi2:
    print("-", p, "| Score:", round(advisor.score(p), 2))

XI built with advisor score: Squad(players=11, cost=75.8/100.0)

Players in XI:
- Roefs (Goalkeeper) - £4.9m | Score: 370.65
- Guéhi (Defender) - £5.2m | Score: 428.38
- Chalobah (Defender) - £5.8m | Score: 420.67
- Mukiele (Defender) - £4.6m | Score: 420.53
- Gabriel (Defender) - £7.1m | Score: 416.01
- Semenyo (Midfielder) - £8.0m | Score: 434.95
- Rice (Midfielder) - £7.6m | Score: 411.81
- Stach (Midfielder) - £4.7m | Score: 389.54
- Anderson (Midfielder) - £5.4m | Score: 386.34
- João Pedro (Forward) - £7.7m | Score: 425.35
- Haaland (Forward) - £14.8m | Score: 396.04


In [40]:
# Test Team_builder
from src.team_builder import build_starting_xi

xi3 = build_starting_xi(player_objects, advisor, formation="4-4-2", budget=100.0)

print("XI from module:", xi3)
for p in xi3:
    print("-", p)

XI from module: Squad(players=11, cost=75.8/100.0)
- Roefs (Goalkeeper) - £4.9m
- Guéhi (Defender) - £5.2m
- Chalobah (Defender) - £5.8m
- Mukiele (Defender) - £4.6m
- Gabriel (Defender) - £7.1m
- Semenyo (Midfielder) - £8.0m
- Rice (Midfielder) - £7.6m
- Stach (Midfielder) - £4.7m
- Anderson (Midfielder) - £5.4m
- João Pedro (Forward) - £7.7m
- Haaland (Forward) - £14.8m
